In [21]:
pip install pytube

     |████████████████████████████████| 46 kB 4.7 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [17]:
import tensorflow as tf
import numpy as np
import cv2
import pytube
import os

from PIL import Image
from skimage import transform

In [2]:
#loading the model
my_model = tf.keras.models.load_model('Model/TheATeam_model_ver2', compile=True)

In [33]:
#Load and pre-process image frames
def load_frames(frame):
   frames = Image.open(frame)
   frames = np.array(frames).astype('float32')/255
   frames = transform.resize(frames, (224, 224, 3))
   frames = np.expand_dims(frames, axis=0)
   return frames

In [44]:
#Downloading real-world ambulance video from youtube
url="https://youtu.be/bnX1JqglJ2E"
yt = pytube.YouTube(url)
pytube.YouTube(url).streams.get_highest_resolution().download('Video Frames')

'/home/jupyter/Video Frames/London Ambulance stuck in traffic !!!.mp4'

In [45]:
#Renaming the video file
path = 'Video Frames/'+yt.title+'.mp4'
os.rename(path ,'Video Frames/testAmbulanceVideo.mp4')

In [47]:
#get video
vidcap = cv2.VideoCapture('Video Frames/testAmbulanceVideo.mp4')

#converting video into frame image (jpg format)
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite("Video Frames/"+str(count)+"_frame.jpg", image)     # save frame as JPG file
        
        image = load_frames("Video Frames/"+str(count)+"_frame.jpg")
        result = my_model.predict(image)
        
        print(str(count)+") Ambulance Detected: {}".format("%.3f" % result[0][1]) if result[0][1]>0.03 
              else str(count)+") Ambulance not detected: {}".format("%.3f" % result[0][1]))
    return hasFrames
sec = 0
frameRate = 5 #//it will capture image in each second
count=1
success = getFrame(sec)

#looping the function
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

1) Ambulance Detected: 0.059
2) Ambulance not detected: 0.001
3) Ambulance Detected: 0.666
4) Ambulance Detected: 0.441
5) Ambulance Detected: 0.543
6) Ambulance Detected: 0.700
7) Ambulance not detected: 0.002
8) Ambulance not detected: 0.006
9) Ambulance not detected: 0.022
10) Ambulance not detected: 0.020
11) Ambulance Detected: 0.088
12) Ambulance not detected: 0.015
13) Ambulance not detected: 0.007
14) Ambulance Detected: 0.597
15) Ambulance Detected: 0.769
16) Ambulance Detected: 0.039
17) Ambulance Detected: 0.092
18) Ambulance Detected: 0.218
19) Ambulance Detected: 0.292
20) Ambulance Detected: 0.170
21) Ambulance Detected: 0.232
22) Ambulance not detected: 0.013
23) Ambulance Detected: 0.163
24) Ambulance Detected: 0.189
25) Ambulance not detected: 0.009
26) Ambulance Detected: 0.385
27) Ambulance Detected: 0.046
28) Ambulance Detected: 0.368
29) Ambulance Detected: 0.861
30) Ambulance Detected: 0.315
31) Ambulance not detected: 0.021
32) Ambulance Detected: 0.059


In [51]:
#See overal result of each categories frrom a frame
image = load_frames('Video Frames/11_frame.jpg')
result = my_model.predict(image)
print(result[0])
print("Ambulance Detected: {}".format(result[0][1]) if result[0][1]>0.3 else "Ambulance not detected: {}".format(result[0][1]))

[4.1202511e-04 8.8441029e-02 6.2943242e-02 2.3630784e-01 6.1189586e-01]
Ambulance not detected: 0.08844102919101715
